In [18]:
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import NamedStyle, Font, PatternFill, Alignment
import os

# Die folgenden imports stammen aus der Datei ../00_common/common.py

import sys
sys.path.append("../00_common")
import common

def check_fingerprint(wb, matr):
    try: 
        ws = wb.worksheets[0]
        if ws["AS1"].value == int(matr):
            return "OK"
        else:
            return "NOK"
    except:
        print("FAIL:", matr, "check_fingerprint")
        return "NOK"

def check_arrayformula(wb, matr):
    try:
        response = "     "
        ws = wb.worksheets[0]
        if ws.array_formulae != {}:
            response = response[:1] + "1" + response[2:]
        
        ws = wb.worksheets[1]
        if ws.array_formulae != {}:
            response = response[:3] + "2" + response[4:]

        return response
    except:
        print("FAIL:", matr, "check Matrix_Formula")
        return "NOK"


def check_exercise_1_1(wb, matr):
    try:
        ws = wb.worksheets[0]
        points = 0
        # Check if background color is FH-Mint
        if ws["B2"].fill.start_color.index == ws["J2"].fill.start_color.index == "FF00B1AC":
            points += 1
        if ws["A1"].font.size == ws["J1"].font.size == 16:
            points += 1
    
        # Check if number-format in column B is Prod-00
        if ws["B10"].number_format == r'"Prod-"00':
            points += 1
    
        return points
    except:
        print("FAIL:", matr, "exercise 1_1")
        return "NOK"

def check_exercise_1_2(wb, wb_val, matr):
    # Check formulas
    try:
        ws = wb.worksheets[0]
        points = 0
        formula = ""
        # Check formulas
        if ws["E20"].value != None:
            E20 = ws["E20"].value
            if "C20" in E20 and "D20" in E20 and ("*" in E20 or "PRODUCT" in E20):
                points += 1
        
        if "C3" in ws.array_formulae:
                formula = ws["C3"].value.text
        else:
            if ws["C3"].value != None:
                formula = ws["C3"].value
        if formula != "":
            formula = formula.split(",")
            if "VLOOKUP" in formula[0] and "B3" in formula[0] and "G" in formula[1]:
                points += 1
            if "VLOOKUP" in formula[0] and "B:B" in formula[0] and "G" in formula[1]:
                points += 1
        
        formula = ""
        if "K3" in ws.array_formulae:
                formula = ws["K3"].value.text
        else:
            if ws["K3"].value != None:
                formula = ws["K3"].value
        if formula != "":
            formula = formula.split(",")
            if "COUNTIF" in formula[0] and "B" in formula[0] and ("J" in formula[1] or "G" in formula[1]):
                points += 1

        formula = ""
        if "L3" in ws.array_formulae:
                formula = ws["L3"].value.text
        else:
            if ws["L3"].value != None:
                formula = ws["L3"].value
        if formula != "":
            formula = formula.split(",")
            if "SUMIF" in formula[0] and "B" in formula[0] and ("J" in formula[1] or "G" in formula[1]) and "D" in formula[2]:
                points += 1
            if "SUMIFS" in formula[0] and "B" in formula[1] and ("J" in formula[2] or "G" in formula[2]) and "D" in formula[0]:
                points += 1
        
        if ws["P2"].value != None:
            if "SUM(" in ws["P2"].value:
                points += 1
            
        if ws["P4"].value != None:
            if "SUMPRODUCT" in ws["P4"].value:
                points += 1
        
        ws_val = wb_val.worksheets[0]
        if ws_val["P2"].value != None and ws_val["P3"].value != None and ws_val["P4"].value != None:
            if round(ws_val["P2"].value) == round(ws_val["P3"].value) == round(ws_val["P4"].value):
                points += 1
           
        return points
    except:
        print("FAIL:", matr, "exercise 1_2")
        return "NOK"
    
def check_exercise_1_3(wb_val, matr):
    # Check if the calculations for Prod-01 in table "Übersicht" are correct
    try:
        ws_val = wb_val.worksheets[0]
        points = 0
        num_prod1 = 0
        turnover_prod1 = 0
        for i in range(3, ws_val.max_row + 1):
            cell = "B" + str(i)
            if ws_val[cell].value == 1:
                cell = "D" + str(i)
                if ws_val[cell].value != None:
                    num_prod1 += ws_val[cell].value
                cell = "E" + str(i)
                if ws_val[cell].value != None:
                    turnover_prod1 += ws_val[cell].value
    
        if ws_val["L3"].value != None:
            if ws_val["L3"].value == num_prod1:
                points += 2
        if ws_val["M3"].value != None:
            if round(ws_val["M3"].value, 2) == round(turnover_prod1, 2):
                points += 2
                
        return points
    except:
        print("FAIL:", matr, "exercise 1_3")
        return "NOK"

def check_exercise_2_1(wb, matr):
    try:
        ws = wb.worksheets[1]
        points = 0
        # Check if background color is FH-Mint
        if ws["B2"].fill.start_color.index == ws["J2"].fill.start_color.index == "FF00B1AC":
            points += 1
        if ws["A1"].font.size == ws["O1"].font.size == 16:
            points += 1
    
        # Check if number-format in column B is Prod-00
        if ws["B10"].number_format == r'"Empl-"000':
            points += 1 
    
        return points
    except:
        print("FAIL:", matr, "exercise 2_1")
        return "NOK"

def check_exercise_2_2(wb, wb_val, matr):
    # Check formulas
    try:
        ws = wb.worksheets[1]
        points = 0
        # Check formulas
        if "G3" in ws.array_formulae:
            if "E3" in ws["G3"].value.text and "F3" in ws["G3"].value.text:
                points += 1
        elif ws["G20"].value != None:
            if ws["G20"].value in ["=E20*F20", "=F20*E20", "=PRODUCT($E20,$F20)", "=PRODUCT(E20,F20)"]:
                points += 1

        # Check Formula in 2nd table. COUNTIFS used correctly?
        formula = ""
        if "J3" in ws.array_formulae:
                formula = ws["J3"].value.text
        else:
            if ws["J3"].value != None:
                formula = ws["J3"].value
        if formula != "":
            formula = formula.split(",")
            if "COUNTIFS" in formula[0] and \
                ("J$2" in formula[1] or "J$2" in formula[3]) and \
                ("$I3" in formula[1] or "$I3" in formula[3]):
                points += 1

        formula = ""
        if "P3" in ws.array_formulae:
                formula = ws["P3"].value.text
        else:
            if ws["P3"].value != None:
                formula = ws["P3"].value
        if formula != "":
            formula = formula.split(",")
            if "SUMIFS" in formula[0] and "$G" in formula[0] and \
                ("P$2" in formula[2] or "P$2" in formula[4]) and \
                ("$O3" in formula[2] or "$O3" in formula[4] or "$O$3" in formula[2] or "$O$3" in formula[4]):
                points += 1

        formula = ""
        if "X3" in ws.array_formulae:
                formula = ws["X3"].value.text
        else:
            if ws["X3"].value != None:
                formula = ws["X3"].value
        if formula != "":
            formula = formula.split(",")
            if "SUMIFS" in formula[0] and "$G" in formula[0] and \
                ("X$2" in formula[2] or "X$2" in formula[4] or "X$2" in formula[6]) and \
                ("$W3" in formula[2] or "$W3" in formula[4] or "$W3" in formula[6] or "$W$3" in formula[2] or "$W$3" in formula[4] or "$W$3" in formula[6]) and \
                ("$V$2" in formula[2] or "$V$2" in formula[4] or "$V$2" in formula[6]) and \
                ("$" in formula[1] and "$" in formula[3]):
                points += 1
        
        return points
    except:
        print("FAIL:", matr, "exercise 2_2")
        return "NOK"

def check_exercise_2_3(wb_val, matr):
    # Check if the calcations for Empl-001, Amsk, North are correct
    try:
        ws_val = wb_val.worksheets[1]
        l_products = []
        empl = ws_val["V2"].value
        values_correct = True
        for i in range (3, 23):
            l_products.append(ws_val["I" + str(i)].value)
        for ind in range(len(l_products)):
            turnover = 0
            for i in range(3, ws_val.max_row + 1):
                if ws_val["B" + str(i)].value == empl:
                    if ws_val["C" + str(i)].value == "NORTH":
                        if ws_val["D" + str(i)].value == l_products[ind]:
                            if ws_val["G" + str(i)].value != None:
                                turnover += ws_val["G" + str(i)].value
            if turnover != ws_val["X" + str(3 + ind)].value:
                values_correct = False
        if values_correct:
            return 5
        else:
            return 0
    except:
        print("FAIL:", matr, "exercise 2_3")
        return "NOK"
    
def check_workbook(matr, wb, wb_val):
    points = [matr]
    points.append(check_fingerprint(wb, matr))
    points.append(check_arrayformula(wb, matr))
    points.append(check_exercise_1_1(wb, matr))
    points.append(check_exercise_1_2(wb, wb_val, matr))
    points.append(check_exercise_1_3(wb_val, matr))
    points.append(check_exercise_2_1(wb, matr))
    points.append(check_exercise_2_2(wb, wb_val, matr))
    points.append(check_exercise_2_3(wb_val, matr))
  
    return points



l_points = common.check_ExcelFiles_in_Abgaben(check_workbook)
for point in l_points:
    print(point)
header = ["MatrNr", "Fingerprint", "Array", "1 Format", "1 Formula", "1 Calc", "2 Format", "2 Formula", "2 Calc"] 
common.result_into_file(l_points, header)


['3668942_1.xlsx']
['~$3642861.xlsx']
[3642861, 'OK', '     ', 2, 7, 4, 3, 1, 5]
[3645619, 'OK', '     ', 3, 7, 4, 3, 4, 5]
[3733682, 'OK', '     ', 3, 7, 4, 3, 4, 5]
[3582059, 'NOK', '     ', 0, 0, 0, 1, 0, 0]
[3714810, 'OK', ' 1   ', 3, 6, 4, 3, 4, 5]
[3647853, 'OK', '     ', 3, 7, 4, 3, 2, 5]
[3659636, 'OK', ' 1   ', 3, 6, 4, 3, 3, 5]
[3711116, 'OK', '     ', 3, 7, 4, 3, 4, 5]
[3558838, 'OK', '     ', 3, 7, 4, 2, 1, 5]
[3699500, 'OK', '     ', 3, 7, 4, 2, 1, 5]
[3663372, 'OK', ' 1   ', 3, 6, 4, 3, 3, 5]
[3652468, 'OK', '     ', 3, 6, 4, 3, 4, 5]
[3718566, 'OK', '     ', 2, 7, 4, 2, 1, 5]
[3731240, 'OK', '     ', 2, 7, 4, 2, 4, 5]
[3727418, 'OK', '     ', 3, 7, 4, 3, 3, 5]
[3533303, 'OK', '     ', 2, 7, 4, 2, 3, 0]
[3735433, 'OK', '     ', 3, 7, 4, 3, 4, 0]
[3582385, 'OK', '     ', 2, 7, 4, 2, 3, 5]
[3732049, 'OK', '     ', 2, 7, 4, 3, 1, 5]
['_3728866.xlsx']
[3589904, 'OK', '     ', 2, 0, 0, 3, 2, 0]
[3590805, 'OK', '     ', 3, 7, 4, 3, 4, 5]
[3710474, 'OK', ' 1   ', 3, 7, 4, 3, 3, 

In [65]:
os.chdir(".")
print(os.getcwd())

/Users/jacobs/GIT/PC_Praktikum/04_Excel
